# Working with Functions To Perform a Service

This notebook shows how you can use functions to perform external tasks such as sending an email. 

## API Configuration

In [1]:
import openai
import json
import os

# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)

# The API key for your Azure OpenAI resource.
openai.api_key = os.getenv("OPENAI_API_KEY")

# Currently Chat Completion API have the following versions available: 2023-07-01-preview
openai.api_version = config_details['OPENAI_API_VERSION']

# This is set to `azure`
openai.api_type = "azure"

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = config_details['OPENAI_API_BASE']

# Setting up the deployment name
deployment_id = config_details['DEPLOYMENT_NAME'] # You need to use the 0613 version of gpt-35-turbo or gpt-4 to work with functions

## Define the function

### Send Email

In [2]:
import smtplib 
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 

def send_email(smtp_server, smtp_port, sender_email, sender_password, recipient_email, subject, body): 
    msg = MIMEMultipart() 
    msg['From'] = sender_email 
    msg['To'] = recipient_email 
    msg['Subject'] = subject 
    msg.attach(MIMEText(body, 'plain')) 

    try: 
        server = smtplib.SMTP(smtp_server, smtp_port) 
        server.starttls()  # Secure the connection 
        server.login(sender_email, sender_password) 
        text = msg.as_string() 
        server.sendmail(sender_email, recipient_email, text) 
        server.quit() 
        return "Email sent successfully!" 

    except Exception as e: 
        return f"Failed to send email: {str(e)}" 

In [3]:
# Define your SMTP server details 
smtp_server = 'smtp.gmail.com' # Replace with your SMTP server this is for gmail 
smtp_port = 587 # Replace with your SMTP server port number this is the port for gmail 

# Define your email details 
sender_email = 'sender@email.com'  # Replace with your email username 
sender_password = 'yourpassword'  # Replace with your email password. For gmail accounts, you need to create an app password 
recipient_email = 'recipient@email.com'  # Replace with recipient's email address 

# Define your email content 
subject = 'Hello!'  # Replace with your subject line 
body = 'This is a test email'  # Replace with your email body text 

# Call the send_email function 
send_email(smtp_server, smtp_port, sender_email, sender_password, recipient_email, subject, body) 

# Steps to create an app password for your gmail account 
# Visit this page: https://myaccount.google.com/security 
# Under the section "2-Step Verification", click on "App passwords".  
# Under "Select app", choose "Mail". 
# Under "Select device", choose the one you're using. 
# Click on "Generate". You will see a 16-digit password. Use this password in your Python script where you'd normally use your Gmail password.

"Failed to send email: (535, b'5.7.8 Username and Password not accepted. Learn more at\\n5.7.8  https://support.google.com/mail/?p=BadCredentials f22-20020a17090ac29600b00263f41a655esm237986pjt.43 - gsmtp')"

## Using GPT to perform a service 
In this example, GPT is used to call a function that sends an email. Before the email is sent, the user must confirm it likes the content generated by the model.

### Helper Method
Checks for invalid arguments.

In [4]:
import inspect

# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided 
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

Replace the user message with your email, your password, the recipient email, the subject, and the body.

In [5]:
def perform_service(): 
    # Step 1: send the conversation and available functions to GPT 
    messages = [] 
    messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous. You are able to help send emails using a function."}) 
    messages.append({"role": "system", "content": "After you generate the content of the email ask the user to confirm the draft before sending it."}) 

    messages = [{"role": "user", "content": "Can you send an email for me?"}] 
    functions = [ 
        { 
            "name": "send_email", 
            "description": "Send an email", 
            "parameters": { 
                "type": "object", 
                "properties": { 
                    "smtp_server": { 
                        "type": "string", 
                        "description": "The SMTP server to use to send the email. For example, smtp.office365.com is the server for outlook accounts, smtp.gmail.com is the server for gmail accounts, and smtp.mail.att.net is for AT&T accounts.", 
                    }, 
                    "smtp_port": { 
                        "type": "integer", 
                        "description": "The SMTP port to use to send the email. For example, 587 is the port for outlook and gmail accounts. 465 is the port for AT&T accounts.", 
                    }, 
                    "sender_email": { 
                        "type": "string", 
                        "description": "The email address of the sender.", 
                    }, 
                    "sender_password": { 
                        "type": "string", 
                        "description": "The password of the sender.", 
                    }, 
                    "recipient_email": { 
                        "type": "string", 
                        "description": "The email address of the recipient.", 
                    }, 
                    "subject": { 
                        "type": "string", 
                        "description": "The subject of the email.", 
                    }, 
                    "body": { 
                        "type": "string", 
                        "description": "The body of the email.", 
                    }, 
                }, 
                "required": ["smtp_server", "smtp_port", "sender_email", "sender_password", "recipient_email", "subject", "body"], 
            }, 
        } 
    ] 
    response = openai.ChatCompletion.create( 
        deployment_id=deployment_id, 
        messages=messages, 
        functions=functions, 
        function_call="auto",  
    ) 
    response_message = response["choices"][0]["message"] 

    print("Response from GPT:") 
    print(response_message) 
    print() 

    # Provide more details to GPT about the email to send
    # Replace the sender email, sender password, and recipient email with the user's actual email addresses
    # Replace the subject and body with whatever you want
    messages.append({"role": "user", "content": "My email is sender@email.com. My password is yourpassword. I am sending an email to recipient@email.com. I want the subject line to be Fun Facts of the Day. For the body of the email I want you to generate fun facts."}) 

    response = openai.ChatCompletion.create( 
        deployment_id=deployment_id, 
        messages=messages, 
        functions=functions, 
        function_call={"name": "send_email"},  
    ) 
    response_message = response["choices"][0]["message"] 

    # Parsing out the generated email parts
    email_args = response_message["function_call"]["arguments"]
    arguments_dict = json.loads(email_args)
    email_subject = arguments_dict["subject"]
    email_body = arguments_dict["body"]
    print("The subject of the email is: " + email_subject)
    print("The body of the email is: " + email_body)
    print()

    # Verifying user wants to send the generated email
    print("Do you want to send this email? (yes/no)")
    user_response = input()
    user_response.lower()

    # Step 2: check if user confirmed they want to call the function
    if response_message.get("function_call"): 
        if user_response == "yes":
            print("Recommended Function call:") 
            print(response_message.get("function_call")) 
            print()  

            # Step 3: call the function 
            # Note: the JSON response may not always be valid; be sure to handle errors 
            available_functions = { 
                "send_email": send_email, 
            }  
            function_name = response_message["function_call"]["name"] 

            # verify function exists 
            if function_name not in available_functions: 
                return "Function " + function_name + " does not exist" 
            fuction_to_call = available_functions[function_name]   

            # verify function has correct number of arguments 
            function_args = json.loads(response_message["function_call"]["arguments"]) 
            if check_args(fuction_to_call, function_args) is False: 
                return "Invalid number of arguments for function: " + function_name 

            # Call the function and return the response 
            function_response = fuction_to_call(**function_args) 
            print(function_response) 
            return function_response 
        else:
            return "Email not sent."
service_response = perform_service() 

Response from GPT:
{
  "role": "assistant",
  "content": "Of course! Please provide me with the necessary details."
}

The subject of the email is: Fun Facts of the Day
The body of the email is: Here are some fun facts for you:

1. A single strand of spaghetti is called a spaghetto.
2. The average person spends about 6 months of their life waiting for red lights to turn green.
3. Only 2% of the world's population has green eyes.
4. The shortest war in history lasted only 38 to 45 minutes.

Enjoy!

Do you want to send this email? (yes/no)
